# Modelos de parendizaje
 
------
Matemáticas para el aprendizaje de maquina

Ana Paulina Castillo Velásquez

30 de mayo 2023
------



## Resumen 

En este cuaderno resolveremos un  problema de caracterización de datos a partir de una frontera de descición lineal con un vector soporte, siendo esto aprendizaje supervisado. Las etiquetas son binarias, es decir tendremos dos: 1,-1. 

La selección del espacio de hipótesis adecuado apra hacer clasificaciones binarias depende de varios factores como la disponibilidad de datos, particularmente se usará el enfoque basado en métodos de aprendizaje supervisado: es posible utilizar algoritmos de clasificación supervisada, como árboles de decisión, bosques aleatorios o máquinas de vectores de soporte (SVM), donde el espacio de hipótesis estaría compuesto por diferentes combinaciones de reglas y umbrales que separan las categorias, utilizando un conjunto de datos de entrenamiento etiquetado.

Se trabajará con SVM y decision tree y los errores serán calculados con validación cruzada; todo esto a partir de la librería sklearn. Se usará un número de particiones igual a 5, esto dado los tiempo de compilamiento del codigo de manera esperimental, fue el número más alto en lograr generar resultados en todos los experiementos en un tiempo menor a 5 minutos.

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [48]:
numero_particiones = 5

## Bases de datos

Importaremos primero la base de datos de la pág [Machine learning repository](https://archive.ics.uci.edu/ml/datasets/banknote+authentication) la cual hace la diferencia entre papel moneda verdadero y falso. se tomaron en cuenta los siguientes parámetros:

1. Varianza de la imagen Wavelet transformada (continua)
2. Asimetría de la imagen Wavelet transformada (continua)
3. Curtosis de la imagen Wavelet Transformada (continua)
4. Entropía de la imagen (continua)

que corresponden a las columnas $0,1,2,3$ respectivamente y la columna $4$ que corresponde a la etiqueta de verdadero o falso.

In [49]:
data = pd.read_csv('data_banknote_authentication.csv', sep=',', header=None) 
data

,0,1,2,3,4
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


In [50]:
y = data[4] # la ultima columna 
y = np.where(y <= 0, -1, 1) #se cambia la etiqueta 0 por -1 para poder aplicar el algoritmo
data.drop(columns = [4], inplace=True) # retiramos la columna de etiquetas
X = data
X = [X.loc[i].tolist() for i in X.index]


#### SVM

Primero veremos como se comporta el método de SVM con validación cruzada. Ajustado el hyper parametro del kernel como lineal a modo de buscar una menor complejidad computacional, además en versiones anteriores se concluyó que la base de datos era linealmente separable por lo cual no es necesario complejizar el modelo.

In [51]:

clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=numero_particiones)
scores


array([0.98909091, 0.98545455, 0.97810219, 1.        , 0.98905109])

In [52]:
accuracy = scores.mean()
desviacion =  scores.std()
error_generalizacion = 1 - accuracy + desviacion
error_generalizacion

0.018733635026232976

Podemos asegurar qué el SVM bajo el problema estipulado tiene a lo más un error de generalización de 0.02 o 2%

#### Desicion Tree

Ahora, consideremos como se comporta el método de DT con validación cruzada. Ajustado el hyper parametro de profundidad a $4$ y hojas maximas por nodo a $2$ dado que es el número de variables usadas y la salida que esparamos es binaria. Esto con el fin de generar una interpretabilidad alta a partir de los valores esperados y el número de variables.

In [59]:

clf_T= DecisionTreeClassifier(max_depth = 4, max_leaf_nodes=2, random_state=0)
scores_T = cross_val_score(clf_T, X, y, cv = numero_particiones)
scores_T


array([0.86181818, 0.82181818, 0.84671533, 0.87956204, 0.8540146 ])

In [54]:

accuracy_T = scores_T.mean()
desviacion_T =  scores_T.std()
error_generalizacion_T = 1 - accuracy_T + desviacion_T
error_generalizacion_T

0.09919344274000999

Este resultado muestra que existe a lo más un 10% de error de generalización en un sistema supremamente simple, lo cual es favorable considerando las limitaciones impuestas

## Base de datos2

Importaremos ahora la base de datos de la pág [Machine learning repository](https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+). Esta base de datos clasifica si un salón está ocupado o no con base a los siguientes  parámetros:

1. Fecha, hora, minuto, milisegundo (Discreta)
2. Temperatura en Celsius (Continua)
3. Humedad relativa en % (Continua)
4. Cantidad de luz (Continua)
5. CO2, en ppm (Continua)
6. Humedad (Continua)
que corresponden a las columnas $0,1,2,3,4,5$ respectivamente y la columna $6$ que corresponde a la etiqueta de verdadero o falso.

Los argumentos para los parametros de los modelos se mantienen.

Importamos la base de datos 

In [55]:
data2 = pd.read_csv('datatraining.csv',  sep=',')
data2.columns = list(range(data2.shape[1]))
data2[0] = data2[0].apply(lambda x: int( (pd.to_datetime(x)).strftime("%Y%m%d%H%M%S") ))
data2.drop(columns = [0], inplace=True)
data2.applymap(lambda x: float(x))


,1,2,3,4,5,6
1,23.18,27.2720,426.0,721.250000,0.004793,1.0
2,23.15,27.2675,429.5,714.000000,0.004783,1.0
3,23.15,27.2450,426.0,713.500000,0.004779,1.0
4,23.15,27.2000,426.0,708.250000,0.004772,1.0
5,23.10,27.2000,426.0,704.500000,0.004757,1.0
...,...,...,...,...,...,...
8139,21.05,36.0975,433.0,787.250000,0.005579,1.0
8140,21.05,35.9950,433.0,789.500000,0.005563,1.0
8141,21.10,36.0950,433.0,798.500000,0.005596,1.0
8142,21.10,36.2600,433.0,820.333333,0.005621,1.0


definimos los nuevos datos a incorporar a los modelos

In [56]:
y2 = data2[6] # la ultima columna 
y2 = np.where(y2 <= 0, -1, 1) #se cambia la etiqueta 0 por -1 para poder aplicar el algoritmo
data2.drop(columns = [6], inplace=True) # retiramos la columna de etiquetas
X2 = data2
X2 = [X2.loc[i].tolist() for i in X2.index]

Veamos ahora SVM

In [57]:

clf2 = svm.SVC( C=1)
scores2 = cross_val_score(clf, X2, y2, cv=numero_particiones)
accuracy2 = scores2.mean()
desviacion2 =  scores2.std()
error_generalizacion2 = 1 - accuracy2 + desviacion2
error_generalizacion2

0.053157063847130044

#### Decision Tree

In [60]:
clf_T2 = DecisionTreeClassifier(max_depth = 5, max_leaf_nodes=2, random_state=0)
scores_T2 = cross_val_score(clf_T2, X2, y2, cv = numero_particiones)
scores_T2
accuracy_T2 = scores_T2.mean()
desviacion_T2 =  scores_T2.std()
error_generalizacion_T2 = 1 - accuracy_T2 + desviacion_T2
error_generalizacion_T2

0.04706277421444057

Aquí vemos errores de generalización de a lo más 5% aproximadamente en ambos modelos.